In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time
import torch.nn as nn
from torchvision import transforms, models

class Food101Dataset(Dataset):
    def __init__(self, data_folder, transform=None):
        self.data_folder = data_folder
        self.transform = transform
        self.classes = sorted(os.listdir(data_folder))
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        self.images = self._load_images()

    def _load_images(self):
        images = []
        for cls_name in self.classes:
            cls_path = os.path.join(self.data_folder, cls_name)
            for img_name in os.listdir(cls_path):
                img_path = os.path.join(cls_path, img_name)
                images.append((img_path, self.class_to_idx[cls_name]))
        return images

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path, label = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

# Define transformation for the images
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Path to your data folder
data_folder = "C:/Users/Vlad Talpiga.VLR_PROJAMZ/OneDrive - Valrom Industrie SRL/Desktop/test_classifier/data"

# Create dataset
food101_dataset = Food101Dataset(data_folder, transform=transform)

# Create dataloader
batch_size = 16
data_loader = DataLoader(food101_dataset, batch_size=batch_size, shuffle=True)

# print(food101_dataset.classes)

# # Iterate through the dataloader for testing
# for images, labels in data_loader:
#     # Here you can feed the images to your model for testing
#     # Replace this with your model inference code
#     print(images[0].shape)
#     print(labels)
#     break  # Stop iteration after the first batch for testing purposes


# see_examples = 5
# for i, (imgs, label) in enumerate(data_loader):
#     clear_output(wait=True)
#     image = imgs[0].numpy().transpose(1, 2, 0)
#     plt.imshow(image)
#     plt.text(5, 15, label[0], fontsize ='xx-large', color='red', fontweight='bold')
#     plt.show()

#     if i >= see_examples - 1:
#       break
#     time.sleep(1)

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
print(torch.cuda.get_device_name(0))

def test_acc(net: nn.Module, test_loader: DataLoader):

  net.to(device)
  net.eval()
  
  total = 0
  correct = 0

  for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)
    total += labels.size(0)

    outputs = net(images)
    _, predicted = torch.max(outputs, 1)
    correct += (predicted == labels).sum().item()

  return correct / total * 100

cuda
NVIDIA GeForce RTX 4060 Laptop GPU


In [4]:
class CustomClassifier(nn.Module):
    def __init__(self, num_classes):
        super(CustomClassifier, self).__init__()

        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.classifier(x)

num_classes = len(food101_dataset.classes)
resnet = models.resnet50(pretrained=True)
resnet = nn.Sequential(*list(resnet.children())[:-1])

for param in resnet.parameters():
    param.requires_grad = False

model = nn.Sequential(
    resnet,
    CustomClassifier(num_classes)
)

In [5]:
start = time.time()
for i in range(1, 21):

    state_dict = torch.load(f'C:/Users/Vlad Talpiga.VLR_PROJAMZ/OneDrive - Valrom Industrie SRL/Desktop/IAVA/Proiect/FoodClassifier/resnet7/resnet50_multiplefclayers_adamw_epoch{i}.pkl')
    model.load_state_dict(state_dict)

    accuracy = test_acc(model, data_loader)

    print(f'ResNet7 - epoch {i}: {accuracy:.2f}%')

print(f'Testing duration: {time.time() - start}')

ResNet7 - epoch 1: 52.16%
ResNet7 - epoch 2: 53.86%
ResNet7 - epoch 3: 56.06%
ResNet7 - epoch 4: 56.90%
ResNet7 - epoch 5: 57.35%
ResNet7 - epoch 6: 56.89%
ResNet7 - epoch 7: 57.54%
ResNet7 - epoch 8: 58.31%
ResNet7 - epoch 9: 58.90%
ResNet7 - epoch 10: 57.43%
ResNet7 - epoch 11: 57.26%
ResNet7 - epoch 12: 58.73%
ResNet7 - epoch 13: 58.27%
ResNet7 - epoch 14: 57.80%
ResNet7 - epoch 15: 56.81%
ResNet7 - epoch 16: 56.39%
ResNet7 - epoch 17: 57.57%
ResNet7 - epoch 18: 59.01%
ResNet7 - epoch 19: 55.84%
ResNet7 - epoch 20: 57.82%
Testing duration: 10935.508822441101


In [6]:
start = time.time()
for i in range(1, 16):

    state_dict = torch.load(f'C:/Users/Vlad Talpiga.VLR_PROJAMZ/OneDrive - Valrom Industrie SRL/Desktop/IAVA/Proiect/FoodClassifier/resnet4/resnet50_epoch{i}.pkl')
    model.load_state_dict(state_dict)

    accuracy = test_acc(model, data_loader)

    print(f'ResNet4 - epoch {i}: {accuracy:.2f}%')

print(f'Testing duration: {time.time() - start}')

ResNet4 - epoch 1: 48.99%
ResNet4 - epoch 2: 51.81%
ResNet4 - epoch 3: 53.76%
ResNet4 - epoch 4: 55.85%
ResNet4 - epoch 5: 57.22%
ResNet4 - epoch 6: 56.31%
ResNet4 - epoch 7: 54.20%
ResNet4 - epoch 8: 57.99%
ResNet4 - epoch 9: 57.74%
ResNet4 - epoch 10: 59.18%
ResNet4 - epoch 11: 57.61%
ResNet4 - epoch 12: 60.06%
ResNet4 - epoch 13: 56.87%
ResNet4 - epoch 14: 59.18%
ResNet4 - epoch 15: 59.46%
Testing duration: 8691.156647443771


In [7]:
start = time.time()
for i in range(1, 16):

    state_dict = torch.load(f'C:/Users/Vlad Talpiga.VLR_PROJAMZ/OneDrive - Valrom Industrie SRL/Desktop/IAVA/Proiect/FoodClassifier/resnet3/resnet50_multiplefclayers_adamw_epoch{i}.pkl')
    model.load_state_dict(state_dict)

    accuracy = test_acc(model, data_loader)

    print(f'ResNet3 - epoch {i}: {accuracy:.2f}%')

print(f'Testing duration: {time.time() - start}')

ResNet3 - epoch 1: 49.39%
ResNet3 - epoch 2: 52.52%
ResNet3 - epoch 3: 55.08%
ResNet3 - epoch 4: 55.05%
ResNet3 - epoch 5: 55.60%
ResNet3 - epoch 6: 55.49%
ResNet3 - epoch 7: 54.63%
ResNet3 - epoch 8: 54.14%
ResNet3 - epoch 9: 58.78%
ResNet3 - epoch 10: 58.36%
ResNet3 - epoch 11: 55.14%
ResNet3 - epoch 12: 56.61%
ResNet3 - epoch 13: 57.45%
ResNet3 - epoch 14: 57.24%
ResNet3 - epoch 15: 56.93%
Testing duration: 8727.28526878357
